# Adopted from https://gitlab.qdatalabs.com/quantiphi-sagemaker-marketplace-examples/bar-qr-code-detection/blob/master/BarcodeQRcodeScannerNB-ARN.ipynb
## Imports

In [1]:
import sagemaker as sage
import os
import boto3
from sagemaker import get_execution_role
from sagemaker import transformer
from sagemaker import model

role = get_execution_role()

## Create the session

The session remembers our connection parameters to Amazon SageMaker. We'll use it to perform all of our SageMaker operations.

In [2]:
sess = sage.Session()

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
print(account)
print(region)

635521884144
us-east-1


## Create Model

Now we use the Bar/QR-code Scanner Model Package to create a model

In [3]:
from src.model_package_arns import ModelPackageArnProvider

In [4]:
from sagemaker import ModelPackage
modelpackage_arn = ModelPackageArnProvider.get_model_package_arn(sess.boto_region_name)

# Define predictor wrapper class
def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session, content_type='image/jpeg')

# Create a deployable model
model = ModelPackage(role=role,
                                      model_package_arn = modelpackage_arn,
                                      sagemaker_session = sess,
                                      predictor_cls = predict_wrapper)
# ARN
print(modelpackage_arn)

arn:aws:sagemaker:us-east-1:865070037744:model-package/bar-qr-code-reader-1571413449-a35a47dda86b16474ecd69ff9f20b46f


### Live Inference Endpoint

Now we demonstrate the creation of an endpoint for live inference

In [5]:
# Deploy the model
predictor = model.deploy(1, 'ml.m5.xlarge', endpoint_name='barcode-endpoint')

-----------!

## Batch Transform Job

Now let's use the model built to run a batch inference job and verify it works.

### Batch Transform Input Preparation


In [6]:
bucket = sess.default_bucket()
print(bucket)

transform_input_prefix = "barcode/validation"
TRANSFORM_WORKDIR = "data/transform" 
# upload data from local directory to bucket

transform_input = sess.upload_data(TRANSFORM_WORKDIR, key_prefix=transform_input_prefix) 

print ("Transform Data Location " + transform_input)

sagemaker-us-east-1-635521884144
Transform Data Location s3://sagemaker-us-east-1-635521884144/barcode/validation


### Run Batch Transform

Now that our batch transform input is setup, we run the transformation job next

In [7]:
transformer = model.transformer(1, 'ml.m5.12xlarge', 
                                output_path="s3://"+bucket+"/barcode/batch-transform-output")
transformer.transform(transform_input, content_type='image/jpeg')
transformer.wait()

print("Batch Transform output saved to " + transformer.output_path)

Using already existing model: bar-qr-code-reader-1571413449-a35a47dda-2020-04-14-12-41-18-288


....................Starting the inference server with 48 workers.
[2020-04-14 12:50:10 +0000] [11] [INFO] Starting gunicorn 19.9.0
[2020-04-14 12:50:10 +0000] [11] [INFO] Listening at: unix:/tmp/gunicorn.sock (11)
[2020-04-14 12:50:10 +0000] [11] [INFO] Using worker: gevent
[2020-04-14 12:50:10 +0000] [15] [INFO] Booting worker with pid: 15
[2020-04-14 12:50:10 +0000] [16] [INFO] Booting worker with pid: 16
[2020-04-14 12:50:10 +0000] [64] [INFO] Booting worker with pid: 64
[2020-04-14 12:50:10 +0000] [65] [INFO] Booting worker with pid: 65
[2020-04-14 12:50:10 +0000] [113] [INFO] Booting worker with pid: 113
[2020-04-14 12:50:10 +0000] [114] [INFO] Booting worker with pid: 114
[2020-04-14 12:50:10 +0000] [178] [INFO] Booting worker with pid: 178
[2020-04-14 12:50:10 +0000] [210] [INFO] Booting worker with pid: 210
[2020-04-14 12:50:10 +0000] [259] [INFO] Booting worker with pid: 259
[2020-04-14 12:50:10 +0000] [308] [INFO] Booting worker with pid: 308
[2020-04-14 12:50:10 +0000] [309

UnexpectedStatusException: Error for Transform job bar-qr-code-reader-1571413449-a35a47dda-2020-04-14-12-46-56-608: Failed. Reason: AlgorithmError: See job logs for more information

In [8]:
# Print the outputs

s3 = boto3.resource('s3')
my_bucket = s3.Bucket(sess.default_bucket())

for object_summary in my_bucket.objects.filter(Prefix="barcode/batch-transform-output"+ "/"):
   file_name = object_summary.key.split('/')[-1]
   print(file_name)
   s3.Bucket(sess.default_bucket()).download_file("barcode/"+"batch-transform-output" +"/"+ file_name,  'batch_results')
   with open('batch_results') as f:
       results = f.readlines()  
   
   print("".join(results))

IMG20200410151016.jpg.out
{"N35E-ZCOO-C1VL-9TL5": "QRCODE"}
IMG20200410151235.jpg.out
{"CW7L-R0WW-JF2K-KD2D": "QRCODE"}
IMG20200410151253.jpg.out
{"J2HY-S0O2-G2HV-8T9D": "QRCODE"}
IMG20200410151309.jpg.out
{"CW7L-R0WW-JF2K-KD2D": "QRCODE", "N35E-ZCOO-C1VL-9TL5": "QRCODE"}
IMG20200410153141.jpg.out
{"N35E-ZCOO-C1VL-9TL5": "QRCODE", "CW7L-R0WW-JF2K-KD2D": "QRCODE", "2888-ABFS-N1IM-D6R0": "QRCODE", "9UIQ-NVR1-DD58-KCYY": "QRCODE", "J2HY-S0O2-G2HV-8T9D": "QRCODE"}
IMG20200410153152.jpg.out
{"N35E-ZCOO-C1VL-9TL5": "QRCODE", "CW7L-R0WW-JF2K-KD2D": "QRCODE"}


### Clean Up Endpoint

In [9]:
predictor.delete_endpoint()